# 02 — Lasso feature discovery (Omitted Variable Bias)

Identify high-impact omitted variables from 1,200+ raw NHGIS columns using **Lasso (L1) regularization**. All screening logic lives in `scripts/advanced_metrics.run_lasso_feature_selection`; this notebook only runs it and displays results.

- **Raw data**: read-only from `data/raw/nhgis/` (no source CSVs modified).
- **Output**: `output/lasso_feature_shortlist.csv` with NHGIS codes, standardized coefficients, and codebook mapping (B25014 Overcrowding, B25070 Rent squeeze prioritized).

In [1]:
import os
import sys
import pandas as pd

REPO_ROOT = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
sys.path.insert(0, os.path.join(REPO_ROOT, "scripts"))
DATA_DIR = os.path.join(REPO_ROOT, "data")
OUTPUT_DIR = os.path.join(REPO_ROOT, "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

from advanced_metrics import run_lasso_feature_selection, NHGIS_CODEBOOK

## Load and examine codebook

Codebook is built from row 2 of each raw NHGIS CSV (variable descriptions). Columns whose description starts with "Margins of error" are excluded from Lasso.

In [2]:
import os
import glob
import pandas as pd

REPO_ROOT = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
DATA_DIR = os.path.join(REPO_ROOT, "data")
raw_nhgis = os.path.join(DATA_DIR, "raw", "nhgis")
if not os.path.isdir(raw_nhgis):
    raw_nhgis = DATA_DIR
nhgis_pattern = os.path.join(raw_nhgis, "nhgis*.csv")
nhgis_files = sorted(glob.glob(nhgis_pattern))

# Build codebook from row 0 (codes) and row 1 (descriptions) of each CSV (same logic as scripts/ingest_nhgis.load_nhgis_codebook)
codebook = {}
exclude_set = set()
for filepath in nhgis_files:
    head = pd.read_csv(filepath, header=None, nrows=2, low_memory=False)
    if head.shape[0] < 2:
        continue
    names = head.iloc[0].astype(str).tolist()
    descs = head.iloc[1].tolist()
    n = min(len(names), len(descs))
    for i in range(n):
        col = names[i]
        desc = descs[i] if pd.notna(descs[i]) else ""
        desc_str = str(desc).strip()
        if col not in codebook:
            codebook[col] = desc_str
        if desc_str.lower().startswith("margins of error"):
            exclude_set.add(col)

# As a DataFrame for easier inspection
codebook_df = pd.DataFrame([
    {"nhgis_code": k, "description": v, "is_margin_of_error": k in exclude_set}
    for k, v in codebook.items()
])
codebook_df = codebook_df.sort_values("nhgis_code").reset_index(drop=True)

# Examine
display(codebook_df)

,nhgis_code,description,is_margin_of_error
0,AIANHHA,American Indian Area/Alaska Native Area/Hawaii...,False
1,AIHHTLI,American Indian/Hawaiian Home Land Trust Land ...,False
2,AITSA,Tribal Subdivision/Remainder Code,False
3,ANRCA,Alaska Native Regional Corporation Code,False
4,AU08E001,Estimates: Total,False
...,...,...,...
1637,TRACTA,Census Tract Code,False
1638,TRUSTA,American Indian Area (Off-Reservation Trust La...,False
1639,UAA,Urban Area Code,False
1640,YEAR,Data File Year,False


## Run Lasso feature selection

Loads raw NHGIS from `data/raw/nhgis/`, uses target **extended_fam_hh_rate** (sum of non-nuclear relatives / householders from B09019). To avoid tautology, **all of Table AU46 (B09019)** — including the target constituents (AU46E003, AU46E018–AU46E023) — are **excluded from the feature set**. Lasso then selects from the **top 100** remaining columns by correlation, runs **LassoCV** (standardized), and writes the **top 30 non-zero** coefficients to `output/lasso_feature_shortlist.csv`. This yields drivers from other tables (e.g. B25014 Overcrowding, B25070 Rent squeeze, B19013 Income) rather than kinship proxies.

In [3]:
from ingest_nhgis import load_raw_nhgis_wide

_out = load_raw_nhgis_wide(DATA_DIR)
df_wide = _out[0] if isinstance(_out, tuple) else _out
df_wide.shape

SLD join: 57,118 / 85,382 tracts with at least one SLD variable


(85382, 947)

In [4]:
# Target: extended_fam_hh_rate (relatives per HH). If missing, restart kernel, re-run load cell, and ensure NHGIS includes B09019 (AU46E003, AU46E018–023).
if "extended_fam_hh_rate" in df_wide.columns:
    df_wide["extended_fam_hh_rate"].describe()
else:
    print("extended_fam_hh_rate not in df_wide. Re-run the load cell above (and restart kernel if needed). NHGIS must include table B09019 (AU46E003 + kinship cols).")

In [11]:
#print summary statistics for multigen_rate
df_wide["extended_fam_hh_rate"].describe()

count    84279.000000
mean         0.187985
std          0.172695
min          0.000000
25%          0.073096
50%          0.138948
75%          0.246146
max          2.913793
Name: extended_fam_hh_rate, dtype: float64

In [12]:
df_wide["AU46E018"].describe()

count    85382.000000
mean        90.951465
std        110.312389
min          0.000000
25%         17.000000
50%         56.000000
75%        125.000000
max       2004.000000
Name: AU46E018, dtype: float64

In [6]:
# Calculate percent missing for each column, sort descending, and display as scrollable DataFrame
import pandas as pd

pd.set_option('display.max_rows', 2000)  # Increase the max number of rows to scroll
missing_pct = (df_wide.isnull().sum() / len(df_wide) * 100)
missing_pct_sorted = missing_pct.sort_values(ascending=False)
display(missing_pct_sorted)

ZCTA5A                  100.000000
SUBMCDA                 100.000000
CBSAA                   100.000000
METDIVA                 100.000000
UAA                     100.000000
CDCURRA                 100.000000
SLDUA                   100.000000
SLDLA                   100.000000
SDELMA                  100.000000
AITSA                   100.000000
SDSECA                  100.000000
SDUNIA                  100.000000
PCI                     100.000000
PUMAA                   100.000000
BTTRA                   100.000000
BTBGA                   100.000000
ANRCA                   100.000000
CSAA                    100.000000
AIHHTLI                 100.000000
AU50E003                100.000000
RES_ONLYA               100.000000
AU50E001                100.000000
AIANHHA                 100.000000
CONCITA                 100.000000
BLKGRPA                 100.000000
REGIONA                 100.000000
PLACEA                  100.000000
DIVISIONA               100.000000
COUSUBA             

In [7]:
results = run_lasso_feature_selection(
    data_dir=DATA_DIR,
    output_path=os.path.join(OUTPUT_DIR, "lasso_feature_shortlist.csv"),
    target_col="extended_fam_hh_rate",
    top_corr_n=100,
    top_nonzero_n=30,
    n_alphas=100,
    cv=5,
)

print(f"Optimal Lasso alpha: {results['optimal_alpha']:.6f}")
print(f"Shortlist written to: {results['output_path']}")
print(f"Top 30 non-zero features: {len(results['shortlist_codes'])}")

SLD join: 57,118 / 85,382 tracts with at least one SLD variable
Optimal Lasso alpha: 0.000106
Shortlist written to: /Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/output/lasso_feature_shortlist.csv
Top 30 non-zero features: 30


## Shortlist: NHGIS codes and standardized coefficients

Exact NHGIS codes with standardized coefficient values. Table **AU46 (B09019)** is excluded from discovery, so the shortlist contains **external drivers** only (no kinship/structure tautology). **Codebook mapping** prioritizes Table **B25014** (Occupants per room / Overcrowding) and **B25070** (Gross rent as % of income / Economic squeeze).

In [8]:
shortlist = results["shortlist"]
display(shortlist)

,nhgis_code,standardized_coef,abs_coef,codebook_table
0,AU46E018,0.074246,0.074246,Estimates: In households: Grandchild
1,AU46E023,0.054838,0.054838,Estimates: In households: Other relatives
2,AU46E020,0.040824,0.040824,Estimates: In households: Parent
3,AU46E019,0.039953,0.039953,Estimates: In households: Brother or sister
4,AUUPE002,0.035792,0.035792,Estimates: Householder who is White alone
5,AUQNE011,-0.029245,0.029245,Estimates: Households with no people under 18 ...
6,AURLE002,-0.026616,0.026616,Estimates: English only
7,AUO7E002,-0.025275,0.025275,Estimates: White alone
8,AU46E022,0.019926,0.019926,Estimates: In households: Son-in-law or daught...
9,AUSYE001,-0.017163,0.017163,Estimates: Per capita income in the past 12 mo...


In [9]:
# Highlight rows that map to B25014 (Overcrowding) or B25070 (Rent squeeze)
b25014 = shortlist["codebook_table"].str.contains("B25014", na=False)
b25070 = shortlist["codebook_table"].str.contains("B25070", na=False)
print("Rows mapping to B25014 (Overcrowding):")
print(shortlist.loc[b25014].to_string())
print()
print("Rows mapping to B25070 (Rent squeeze):")
print(shortlist.loc[b25070].to_string())

Rows mapping to B25014 (Overcrowding):
Empty DataFrame
Columns: [nhgis_code, standardized_coef, abs_coef, codebook_table]
Index: []

Rows mapping to B25070 (Rent squeeze):
Empty DataFrame
Columns: [nhgis_code, standardized_coef, abs_coef, codebook_table]
Index: []


## Top 3 for schema integration

Copy the **nhgis_code** values below into `scripts/core_metrics.py`: add them to `ANALYSIS_READY_SCHEMA["feature_cols"]` and add human-readable labels to `FEATURE_LABELS`. Raw NHGIS columns are preserved in the wide merge, so these codes will be available in analysis-ready data when included in the schema.

In [10]:
top3 = shortlist.head(3)
print("Top 3 Lasso-selected variables (add to core_metrics.py):")
for _, row in top3.iterrows():
    print(f"  {row['nhgis_code']}: {row['codebook_table']}  (std coef = {row['standardized_coef']:.4f})")

Top 3 Lasso-selected variables (add to core_metrics.py):
  AU46E018: Estimates: In households: Grandchild  (std coef = 0.0742)
  AU46E023: Estimates: In households: Other relatives  (std coef = 0.0548)
  AU46E020: Estimates: In households: Parent  (std coef = 0.0408)
